In [1]:
import numpy as np
import pandas as pd
import mrcfile
import cv2
import glob
from matplotlib import pyplot as plt, patches

import math
%matplotlib inline
from moviepy.Clip import *
import os
import csv
import json
import argparse
import numpy as np

In [2]:
phase = 'train'
output_dir = '/bml/ashwin/ViTPicker/train_val_test_data/annotations/'
mrc_file_path = '/bml/ashwin/ViTPicker/train_val_test_data/train/'
coord_file_path  = '/bml/ashwin/ViTPicker/train_val_test_data/particle_coordinates/'



In [3]:
def create_json_dataset(phase, mrc_file_path, coord_file_path, output_dir):
    dataset = {'info': [], 'categories': [] , 'images': [],  'annotations': []}
    classes = ['particle']
    csv_file_not_count = 0
    anno_id = 1

    dataset['info'].append({
        'description': 'coco dataset for Cryo EM micrographs', 
        'contributor': 'Ashwin', 
        'year': '2023'})

    for i, cls in enumerate(classes, 1):
        dataset['categories'].append({'id': i, 'name': cls, 'supercategory': ''})

    file_names = [f for f in sorted(os.listdir(mrc_file_path))]

    for image_id, file_name in enumerate(file_names):
        # write image id, name, width and height
        image_path = mrc_file_path + file_name
        read_image = cv2.imread(image_path)
        IMG_HEIGHT, IMG_WIDTH, channel = read_image.shape
        dataset['images'].append({
                          'id': image_id,
                          'file_name': file_name[:-4] + '.jpg',
                          'width': IMG_WIDTH,
                          'height': IMG_HEIGHT})

        #read particle coordinates
        particle_coord_path = os.path.join(coord_file_path, '') + file_name[:-4] + '.csv'
        if not os.path.exists(particle_coord_path):
            print(f"coordinates not avaibale for {file_name} ----> Skipping Image ID : {image_id}")
            dataset['annotations'].append({
                'id': anno_id,  #annotation id of its own
                'category_id': 1,  # particle class
                'iscrowd': 0,
                'area': 0,
                'image_id': image_id,
                'bbox': [0,0,0,0],
                'segmentation': []
            })
            anno_id += 1
            csv_file_not_count += 1
        
        else:
            boxes = pd.read_csv(particle_coord_path, usecols=[0,1,2])
            for i, box in boxes.iterrows():
                box_x_coord = int(box['X-Coordinate'] - box['Diameter'] / 2 )#top left x coord
                box_y_coord = int(box['Y-Coordinate'] - box['Diameter'] / 2 ) #top left y coord
                BOX_WIDTH = int(box['Diameter'])  

                dataset['annotations'].append({
                    'id': anno_id,  #annotation id of its own
                    'category_id': 1,  # particle class
                    'iscrowd': 1,
                    'area': BOX_WIDTH * BOX_WIDTH,
                    'image_id': image_id,
                    'bbox': [box_x_coord, box_y_coord, BOX_WIDTH, BOX_WIDTH],
                    'segmentation': []
                })
                anno_id += 1


    print("------------------------  STATS   ---------------------")
    print(f"Total Micrographs : {len(file_names)}")
    print(f"CSV file not available for : {csv_file_not_count} micrographs")
    print("------------------------     ---------------------")

    # save json annotation results
    json_name = os.path.join(output_dir, 'instances_{}.json'.format(phase))
    print("json_name:", json_name)
    with open(json_name, 'w') as f:
        json.dump(dataset, f)

In [4]:
create_json_dataset(phase, mrc_file_path, coord_file_path, output_dir)


------------------------  STATS   ---------------------
Total Micrographs : 10
CSV file not available for : 0 micrographs
------------------------     ---------------------
json_name: /bml/ashwin/ViTPicker/small_10_3_json_data_no_coord_missing/annotations/instances_train.json
